In [ ]:
import wikipedia
import pandas as pd
import numpy as np
import re


Make function for wiki pages. 

In [ ]:
import wikipedia
import requests
import urllib.parse
from bs4 import BeautifulSoup
from ratelimit import limits, sleep_and_retry
from tqdm import tqdm
URL = "https://randomincategory.toolforge.org/Random_page_in_category?"
URL += f"category={urllib.parse.quote('famous fictional characters')}"
URL += f"&category2={urllib.parse.quote('historical figures')}"
URL += f"&category3={urllib.parse.quote('slogans')}"
URL += f"&category4={urllib.parse.quote('catchphrases')}"
URL += f"&category5={urllib.parse.quote('authors')}"
URL += f"&category6={urllib.parse.quote('actors')}"
URL += f"&category7={urllib.parse.quote('british authors')}"
URL += f"&category8={urllib.parse.quote('american authors')}"
URL += f"&category9={urllib.parse.quote('musicians')}"
URL += f"&category10={urllib.parse.quote('songs')}"
URL += f"&category11={urllib.parse.quote('books')}"
URL += f"&category12={urllib.parse.quote('movies')}"
URL += "&server=en.wikipedia.org&cmnamespace=&cmtype=page&returntype="


replacements = {
    r'\bsex\b': 'affection',
    r'\bporn\w*\b': 'adult entertainment',
    r'fuck': 'have intimate relations with',
    r'fucking': 'romancing, physically',
    r'\bfuck\w*\b': ' (bleep) ',
    r'\bass\b': 'dump truck',
    r'\bshit\w*\b': 'poop',
    r'damn\w*\b': 'darn',
    r'god-damn\w*\b|goddamn\w*\b': 'super darn',
    r'\bass\b|\wasse*': 'rear end',
    r'cock': 'rooster',
    r'small dick energy':'little man syndrome',
    r'\bdick\w*\b': ' johnson ',
    r'ppl':'people',
    '\bwat\b':'what',
    r'a black': 'an african-american',
    r'black person':'dark skinned person',
    r'\bpee\b' : 'urinate',
    r'\bpiss\b': 'urinate',
    r'\b(peeing|pissing)': 'urinating',
    r'white person':'caucasian',
    r'\bwhore\w*\b': 'prostitute',
    r'nigg*': 'racial slur',
    '\r\n':' ',
    r'\bslut\b': ' loose person ',
    r'\bblowjob\b': 'random gift',
    r'racist': 'ignorant',
    'racism':'fear of people that look different',
    r'faggot': 'gay person',
    r'\bfag\w*\b':'gay person',
    r'boob|boob*|breast': 'lady pillows',
    r'\bbitch*\b': 'mean woman',
    r'bastard*': 'illegitimate child',
    r'hoes?': 'chicks',
    r'breast*|jugs': 'chest',
    r'\bcunt*\b': 'comtemptible person',
    r'\bpuss\w*\b': 'vagina',
    r'\wdick*': 'penis',
    r'naked': 'disrobed',
    r'\bnud\w*\b': 'unclothed',
    r'\nmasterbate\w*\b': 'self-gratified',
    r'\bmasturbating\w*\b': 'gratifying themselves',
    r'\b\w{4}ilf\b': 'person id like to get to know',
    r'mastu\w*\b': 'self-gratification',
    'god':'deity',
    'jesus':'religious figure',
    'christ':'religious figure',
    'bible':'religious text',
    'church':'place of worship',
    'religion':'set of beliefs',
    r'\bpray\w*\b':'communicate with a deity',
    'prayer':'a conversation with a deity',
    'faith':'belief',
    ' lord ':'captain',
    ' allah ':'a diety',
    'gay':'homosexual',
    r' rapist ': ' intense toucher ',
    r' rape ': ' unwelcomed tickling ', # eek I know... 
    r'pedo\w*\b': 'seventies mustached van driver',
    'yahwey':'a deity',
    'yeshua':'a religious figure',
    ' sexual':' reproductive',
    'douche':'chad',
    ' sex ':' private adult time ',
    'milf':"mother I would like to take on a date",
    ' butt ':"dumptruck/tailgate"
}

def replacer_censor(definition,phrase,replacements_dict):
    # Iterate over the keys in the replacements dictionary
    for pattern in replacements_dict:
        # Use re.sub to replace the occurrences of the pattern with its corresponding value in the phrase and definition strings
        phrase = re.sub(pattern, replacements_dict[pattern], phrase)
        definition = re.sub(pattern, replacements_dict[pattern], definition)
    return phrase, definition


def unpack_definitions(phrase,definition):
    # remove the brackets and clean up the definitions
    # with regex
    definition = definition.replace("[","")
    definition = definition.replace("]","")
    definition = definition.replace("'","")
    definition = definition.replace('"',"")
    definition = definition.replace("(","")
    definition = definition.replace(")","")

    # remove double spaces
    definition = definition.replace('  ',' ')
    # also remove any .. by replacing them with a single .
    definition = definition.replace('..','.')
    # remove all non-ascii characters
    definition = re.sub(r'[^\x00-\x7f]',r'', definition)
    # remove any words that are not in the english dictionary
    #english_words = set(w.lower() for w in nltk.corpus.words.words())
    #definition = re.sub(r'\b\w+\b', lambda m: m.group(0) if m.group(0) in english_words else '', definition)
    # remove extra spaces
    #definition = re.sub(' +', ' ', definition)
    
    phrase, definition = replacer_censor(definition,phrase,replacements)
    

    return phrase, definition
# limit to 1 request per second
@sleep_and_retry
def get_random_wiki_entry():

    #print("getting random wiki entry from url: ",URL)
    # use requests to get the page, then pass the redirected page url to wikipedia library.
    req_url = requests.get(URL).url # this is a random page from the category
    # using wikipedia library
    page = wikipedia.page(req_url) # this is a random page from the category
    title = page.title
    summary = page.summary
    #categories = page.categories
    related = page.links
    # create a dictionary of the entry
    random_wiki_entry_dict = {
        "title": title,
        "summary": unpack_definitions(title,summary),
        "related": related,
    }

    return random_wiki_entry_dict

# make a card from the random wiki entry
def create_ppn_card():
    # create a person/place/thing card
    # get a random wikipedia entry
    random_wiki_entry_dict = get_random_wiki_entry()
    # get the summary of the entry
    random_wiki_entry_summary = random_wiki_entry_dict["summary"]
    # get the title of the entry
    random_wiki_entry_title = random_wiki_entry_dict["title"]
    # get the categories of the entry
    random_wiki_entry_related = random_wiki_entry_dict["related"]

    # create a dictionary of the entry
    random_wiki_entry_dict = {
        "title": random_wiki_entry_title,
        "summary": random_wiki_entry_summary,
        "related": len(random_wiki_entry_related)
    }

    return random_wiki_entry_dict

# create a deck of fifty cards from the random wiki entries (non-repeating) and return the deck as a list of dictionaries

def make_one_card():
    while True:
        try:
            card = create_ppn_card() # create a card
        except Exception as e:
            continue
        else:
            if len(card["summary"][1]) > 10 and 'Wiki' not in card["title"]: # if the card is valid
                break
            else:
                continue
    return card

def create_ppn_deck(card_count=50):
    # create a deck of card_count cards
    ppn_deck = []
    for i in tqdm(range(card_count)):
        ppn_deck.append(make_one_card())
    return ppn_deck


In [ ]:
card_deck = create_ppn_deck(50) # create a deck of 50 cards
# show the deck

# Summarization

In [ ]:

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
# import LexRankSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
def summarize_text(text, num_sentences):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, num_sentences)
    return summary

text = "This is some text that I want to summarize. It can be as long or as short as needed. The summary will be a few sentences long."
num_sentences = 2
summary = summarize_text(text, num_sentences)

for sentence in summary:
    print(sentence)


# go through the deck and summarize the text in the summary field of each card, then add the summary to the card's dictionary as a new field called "summary_short"
sentence_count = 2
for card in card_deck:
    summary = summarize_text(card["summary"][1], int(sentence_count))
    summary_short = ""
    for sentence in summary:
        summary_short += str(sentence)
    card["summary_short"] = summary_short

# show the deck
print(f'I have created a deck of {len(card_deck)} cards')


import sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.sum_basic import SumBasicSummarizer
from sumy.summarizers.luhn import LuhnSummarizer

# These are the available summarization methods in sumy
summarizers = [
    "LsaSummarizer", 
    "LexRankSummarizer", 
    "KLSummarizer", 
    "TextRankSummarizer", 
    "SumBasicSummarizer",
    "LuhnSummarizer",
]

# This function demonstrates how to use each summarization method
def demonstrate_summarizers(text, language="english"):
    parser = PlaintextParser.from_string(text, Tokenizer(language))
    for summarizer in summarizers:
        summarizer = eval(summarizer)() # create an instance of the summarizer
        summary = summarizer(parser.document, 3) # summarize the document with 3 sentences
        print("====== Example: ", summarizer," ======")

        for sentence in summary:
            print(sentence)
        print()
    

text = "Reddit ; stylized in all lowercase as reddit is an American social news aggregation, content rating, and discussion website. Registered users commonly referred to as Redditors submit content to the site such as links, text posts, images, and videos, which are then voted up or down by other members. Posts are organized by subject into user-created boards called communities or subreddits. Submissions with more upvotes appear towards the top of their subreddit and, if they receive enough upvotes, ultimately on the sites front page. Reddit administrators moderate the communities. Moderation is also conducted by community-specific moderators, who are not Reddit employees.As of March 2022, Reddit ranks as the 9th-most-visited website in the world and 6th most-visited website in the U.S., according to Semrush. About 4249.3% of its user base comes from the United States, followed by the United Kingdom at 7.98.2% and Canada at 5.27.8%. "
demonstrate_summarizers(text)


!pip install pyenchant

In [ ]:
# clean the text of the summary field of each card by removing all special characters, replacing newlines with spaces, numbers with their word equivalent, and removing all words that are not in the english dictionary.

import re
from nltk.corpus import words
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
# import LexRankSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
# create a list of english words
english_words = words.words()

# go through the deck and summarize the text in the summary field of each card, then add the summary to the card's dictionary as a new field called "summary_short"
sentence_count = 2
for card in card_deck:
    summary = summarize_text(card["summary"][1], int(sentence_count))
    summary_short = ""
    for sentence in summary:
        summary_short += str(sentence)
    card["summary_short"] = summary_short

# show the deck
print(f'I have created a deck of {len(card_deck)} cards')


correct_cards = 0
while len(card_deck) < 50:
    # create a deck of fifty cards from the random wiki entries (non-repeating) and return the deck as a list of dictionaries
    card_deck = create_ppn_deck(50) # create a deck of 50 cards
    
    # go through the deck and summarize the text in the summary field of each card, then add the summary to the card's dictionary as a new field called "summary_short"
    sentence_count = 2
    for card in card_deck:
        summary = summarize_text(card["summary"][1], int(sentence_count))
        summary_short = ""
        for sentence in summary:
            summary_short += str(sentence)
        card["summary_short"] = summary_short

    # remove any cards with words in the title that are in the banned words list
    banned_words = ['List']

    for card in card_deck:
        for word in banned_words:
            if word in card["title"]:
                card_deck.remove(card)

    # clean the text of the summary field of each card by removing all special characters, replacing newlines with spaces, numbers with their word equivalent, and removing all words that are not in the english dictionary.
    for card in card_deck:
        card["summary_clean"] = re.sub(r"[^a-zA-Z0-9]+", ' ', card["summary_short"])
        card["summary_clean"] = card["summary_clean"].replace("\n", " ")

    # if a card has more than 1/4 of its words in the english dictionary, then it is valid, else drop the card
    for card in card_deck:
        card["summary_clean"] = word_tokenize(card["summary_clean"])
        card["summary_clean"] = [word for word in card["summary_clean"] if word in english_words]
        if len(card["summary_clean"]) < len(card["summary_short"])/4:
            card_deck.remove(card)

    # use unpack_definitions on the summary_clean field to remove unusual bad words
    for card in card_deck:
        card["summary_clean"] = unpack_definitions(card['title'],card["summary_clean"])


    # show the deck
    print(f'I have created a deck of {len(card_deck)} cards')


    # truncate all page summaries to be no more than 1200 characters, ending at the nearest sentence boundary. This is to prevent the summary from being too long for the flashcard.
    for card in card_deck:
        summary = card["summary_short"]
        if len(summary) > 1200:
            summary = summary[:1200]
            summary = summary[:summary.rfind('.')+1]
            card["summary_short"] = summary

    # drop all cards with a summary that is less than 100 characters long
    card_deck = [card for card in card_deck if len(card["summary_short"]) > 100]

    # use LexRankSummarizer to summarize the text in the summary field of each card, then add the summary to the card's dictionary as a new field called "summary_short"
    for card in card_deck:
        if card['status'] == 'good':
            continue
        summarizer = LexRankSummarizer() # create an instance of the summarizer
        parser = PlaintextParser.from_string(card["summary"][1], Tokenizer("english"))
        summary = summarizer(parser.document, 3) # summarize the document with 3 sentences
        summary_short = ""
        for sentence in summary:
            summary_short += str(sentence)
        card["summary_short"] = summary_short
        card['status'] = 'good'


In [ ]:
print(f'I have created a deck of {len(card_deck)} cards')
for card in card_deck:
    print(card["title"])
    print(card["summary_short"])
    print(card["related"])
    print()

In [ ]:
# save the deck to a json file by combining the card dictionaries into a json file format.

import json

with open('ppn_deck.json', 'w') as outfile:
    json.dump(card_deck, outfile, indent=4)


In [ ]:
import nltk

deck = [{
        "title": "Neue Volksmusik",
        "summary": [
            "Neue Volksmusik",
            "Neue Volksmusik sometimes also called Volxmusik or Tradimix; English for New folk music describes the crossover mix of traditional German folk music Volksmusik with newer genres such as jazz, contemporary folk, electronic music, and/or rock."
        ],
        "related": 23
    }]

card = deck[0]
title = card["title"]
summary = card["summary"][1]

# Split the summary into a list of sentences
sentences = nltk.sent_tokenize(summary)

# Split the title into a list of words
title_words = title.split()

# Iterate through the sentences and reword if necessary
reworded_sentences = []
for sentence in sentences:
    # Check for words from the title in the sentence
    for word in title_words:
        if word in sentence:
            # Reword the sentence to avoid using the word
            sentence = sentence.replace(word, "<word>")
    reworded_sentences.append(sentence)

# Join the reworded sentences into a single string
reworded_summary = " ".join(reworded_sentences)
print(reworded_summary)

In [ ]:
import nltk
from nltk.corpus import wordnet
nltk.download("wordnet")

In [ ]:
import nltk
from nltk.corpus import wordnet

deck = [{
        "title": "Get Brexit Done",
        "summary": [
            "Get Brexit Done",
            "The slogan 'Get Brexit Done' was used by the Conservative Party in the 2019 United Kingdom general election. It was intended to convey the party's commitment to taking the UK out of the European Union."
        ],
        "related": 23
    }]

card = deck[0]
title = card["title"]
summary = card["summary"][1]

# Split the summary into a list of sentences
sentences = nltk.sent_tokenize(summary)

# Split the title into a list of words
title_words = title.split()

# Iterate through the sentences and reword if necessary
reworded_sentences = []
for sentence in sentences:
    # Check for words from the title in the sentence
    for word in title_words:
        if word in sentence:
            # Reword the sentence to avoid using the word
            sentence = sentence.replace(word, "this (slogan)")
    reworded_sentences.append(sentence)

# Join the reworded sentences into a single string
reworded_summary = " ".join(reworded_sentences)
print(reworded_summary)

In [ ]:
import random
import wikipedia
import json
@sleep_and_retry
def get_page_from_wiki(title):
    return wikipedia.page(title)

def page_is_person(page):
    # people pages have the words "birth" "family" and potentially "death" in the article text (birth and death are not always present). We can use this to filter out people pages.
    if "birth" in page.content.lower() and "family" in page.content.lower():
        return True
    else:
        return False

def page_is_location(page):
    # location pages have the words "location" "country" and "city" in the article text. We can use this to filter out location pages.
    if "location" in page.content.lower() and "country" in page.content.lower() and "city" in page.content.lower():
        return True
    elif page.coordinates is not None:
        return True
    else:
        return False

def page_is_organization(page):
    # organization pages have the words "organization" "company" and "business" in the article text. We can use this to filter out organization pages.
    if "organization" in page.content.lower() or "company" in page.content.lower() and "business" in page.content.lower():
        # also cannot be a person or location
        if not page_is_person(page) and not page_is_location(page):
            return True
        else:
            return False
    else:
        return False

def page_is_event(page):
    # event pages have the words "event" "incident" and "disaster" in the article text. We can use this to filter out event pages.
    if "event" in page.content.lower() and "date" in page.content.lower() and "history" in page.content.lower():
        if not page_is_person(page) and not page_is_location(page) and not page_is_organization(page):
            return True
        else:
            return False
    else:
        return False


def generate_related_deck(primary_card, number_of_cards_to_generate, min_len=100, min_links=5, people=True,locations=True,organizations=True,events=True,other=False):
    new_card_deck = []
    primary_card_title = primary_card['title']
    primary_card_related = wikipedia.page(primary_card_title).links
    related_pages = [page for page in primary_card_related]
    random.shuffle(related_pages)
    related_pages = [page for page in related_pages if page not in [card['title'] for card in new_card_deck]]

    # Loop through each related page
    for random_page in related_pages:
        try:
            page = get_page_from_wiki(random_page)
            random_wiki_entry_summary = page.summary
            random_wiki_entry_title = page.title
            random_wiki_entry_related = page.links
            random_wiki_entry_dict = {
                "title": random_wiki_entry_title,
                "summary": random_wiki_entry_summary,
                "related": len(random_wiki_entry_related),
            }
            if random_wiki_entry_dict['title'] not in [card['title'] for card in new_card_deck]:
                
                # check for min length
                if len(random_wiki_entry_dict['summary']) < min_len:
                    continue
                # check for min links
                if random_wiki_entry_dict['related'] < min_links:
                    continue
                
                # check for people, locations, organizations, events, and other categories in the related pages. Follow the logic outlined by T/F in the function arguments. If people is true for example, then we will add the page if it is a person. Else we ignore people pages.
                # people pages have the words "birth" "family" and potentially "death" in the article text (birth and death are not always present). We can use this to filter out people pages.
                # locations have a page.coordinates attribute that is not None
                # organizations - ?
                # events - have a date in the article text (not always present)?
                
                is_person = page_is_person(page)
                is_location = page_is_location(page)
                is_organization = page_is_organization(page)
                is_event = page_is_event(page)
                print(page.title)
                print(f'-'*8)
                print(f'is_person: {is_person}')
                print(f'is_location: {is_location}')
                print(f'is_organization: {is_organization}')
                print(f'is_event: {is_event}')
            
                if people and is_person:
                    new_card_deck.append(random_wiki_entry_dict)
                    print(f'Added {random_wiki_entry_dict["title"]} to the deck')
                elif locations and is_location:
                    new_card_deck.append(random_wiki_entry_dict)
                    print(f'Added {random_wiki_entry_dict["title"]} to the deck')
                elif organizations and is_organization:
                    new_card_deck.append(random_wiki_entry_dict)
                    print(f'Added {random_wiki_entry_dict["title"]} to the deck')
                elif events and is_event:
                    new_card_deck.append(random_wiki_entry_dict)
                    print(f'Added {random_wiki_entry_dict["title"]} to the deck')
                elif other and not is_person and not is_location and not is_organization and not is_event:
                    new_card_deck.append(random_wiki_entry_dict)
                    print(f'Added {random_wiki_entry_dict["title"]} to the deck')
                else:
                    print(f'Not adding {random_wiki_entry_dict["title"]} to the deck')
                # new_card_deck.append(random_wiki_entry_dict)
                # print(f'Added {random_wiki_entry_dict["title"]} to the deck')
                print(f'{primary_card_title} >> {random_wiki_entry_dict["title"]}')
            with open('./new_ppn_deck.json', 'w') as outfile:
                json.dump(new_card_deck, outfile, indent=4)
            # Break the loop if the number of cards in the deck reaches the desired number
            #print(f'Number of cards in deck: {len(new_card_deck)}/{number_of_cards_to_generate}')
            if len(new_card_deck) >= number_of_cards_to_generate:
                #print("We have enough cards. Exiting loop.")
                break
        except Exception as e:
            continue
    return card_deck


In [ ]:
# test the function by generating a deck of 10 cards that are related to the card "Get Brexit Done"
new_card_deck = []
with open('./new_ppn_deck.json') as json_file:
    new_card_deck = json.load(json_file)

# get the first card in the deck
#primary_card = new_card_deck[0]

primary_card = {
        "title": "Get Brexit Done",
        "summary": [
            "Get Brexit Done",
            "Get Brexit Done was a political slogan frequently used by the British Conservative Party and former Prime Minister Boris Johnson in the run up to the 2019 general election. The slogan reflected the partys pledge to, if re-elected, facilitate the United Kingdoms withdrawal from the European Union by the end of January the following year."
        ],
        "related": 328
    }


# generate a deck of 10 cards that are related to the primary card
new_deck = generate_related_deck(primary_card,50)

# save the deck to a json file by combining the card dictionaries into a json file format.
import json
# save the deck to a json file by combining the card dictionaries into a json file format.

with open('./new_ppn_deck.json', 'w') as outfile:
    json.dump(new_card_deck, outfile, indent=4)



---

In [157]:
import requests
# import access token from secrets.json
with open('./secrets.json') as json_file:
    secrets = json.load(json_file)
    access_token = secrets['GROUPME_API_ACCESSTOKEN']
    group_id = secrets['GROUP_NUMBER_MONIKERS']
def send_message(group_id, message_text, access_token):
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    data = {
        "message": {
            "text": message_text
        }
    }
    response = requests.post(f"https://api.groupme.com/v3/groups/{group_id}/messages", headers=headers, json=data)
    if response.status_code != 201:
        print(f"Failed to send message: {response.status_code} {response.text}")
    else:
        print("Message sent successfully.")


In [165]:
import requests

# Replace :id with the ID of the group you want to join
# Replace :share_token with the share token of the group you want to join

# Import access token from secrets.json
with open('./secrets.json') as json_file:
    secrets = json.load(json_file)
    access_token = secrets['GROUPME_API_ACCESSTOKEN']
    group_id = secrets['GROUP_NUMBER_MONIKERS']
    share_token = secrets['share_token']


# Set the headers for the request
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Make the POST request to the API
group_url = 'https://groupme.com/join_group/91111382/up1X7UZw'

# Make the POST request to the API
response = requests.post(group_url, headers=headers)

# Check the status code of the response
if response.status_code != 200:
    print(f"Failed to make request: {response.status_code} {response.text}")
else:
    print("Request successful.")


Request successful.
Request successful.
<!DOCTYPE html>
<html lang="en-US">
	<head>
		<link rel="icon" type="image/png" sizes="16x16" href="../../favicon-16x16.png">
		<link rel="icon" type="image/png" sizes="32x32" href="../../favicon-32x32.png">

		<meta charset="utf-8" />
		<meta name="viewport" content="width=device-width" />
		<meta name="og:site_name" content="GroupMe">
		<meta name="og:type" content="website">
		<meta name="og:url" content="https://groupme.com">
		<meta name="og:title" content="GroupMe | Where your group chats best">
		<meta name="og:description" content="The easy way to connect with all the groups in your life, big and small">
		<meta name="og:image" content="https://cdn.groupme-b.com/images/og_image_poundie.png">
		<meta name="twitter:card" content="summary_large_image">
		<meta name="twitter:site" content="GroupMe">
		<meta name="twitter:url" content="https://groupme.com">
		<meta name="twitter:title" content="GroupMe | Where your group chats best">
		<meta n

In [172]:
import requests

# Replace :bot_id with your bot's ID
bot_id = "c9d6f93a4e96eeadbeaf21feef"

# Set the text of the message you want to send
message_text = "Welcome to the ever expanding world of Generative Monikers! I am your host, Hubert."

# Set the payload for the request
payload = {
    "bot_id": bot_id,
    "text": message_text
}

# Make the POST request to the GroupMe API
response = requests.post("https://api.groupme.com/v3/bots/post", json=payload)

# Check the status code of the response
if response.status_code != 202:
    print(f"Failed to send message: {response.status_code} {response.text}")
else:
    print("Message sent successfully.")
    
    # Extract the member data from the response
    member_data = response.json()['response']
    
    # Create a list of member IDs
    member_ids = [member['user_id'] for member in member_data]
    
    # Print the list of member IDs
    print(member_ids)


Message sent successfully.


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [175]:
import requests

def groupme_bot():
    # Replace :bot_id with your bot's ID
    # read bot_id from secrets.json
    with open('./secrets.json') as json_file:
        secrets = json.load(json_file)
        bot_id = secrets['groupme_botid']
    
    # Set the text of the message you want to send
    message_text = "Welcome to the ever expanding world of Generative Monikers! I am your host, Hubert."

    # Set the payload for the request
    payload = {
        "bot_id": bot_id,
        "text": message_text
    }

    # Make the POST request to the GroupMe API
    response = requests.post("https://api.groupme.com/v3/bots/post", json=payload)

    # Check the status code of the response
    if response.status_code != 202:
        print(f"Failed to send message: {response.status_code} {response.text}")
    else:
        #print("Message sent successfully.")
        pass
        # # Extract the member data from the response
        # member_data = response.json()['response']
        
        # # Create a list of member IDs
        # member_ids = [member['user_id'] for member in member_data]
        
        # # Print the list of member IDs
        # print(member_ids)


# Google Forms

In [176]:
!pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.0/177.0 kB 318.7 kB/s eta 0:00:0000:0100:01
  Using cached google_auth_httplib2-0.1.0-py2.py3-none-any.whl (9.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 28.2 MB/s eta 0:00:0000:010:01
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.8/96.8 kB 12.1 MB/s eta 0:00:00
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.6.0
    Uninstalling google-auth-2.6.0:
      Successfully uninstalled google-auth-2.6.0


In [ ]:
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Replace :form_id with the ID of your Google Form
form_id = ":form_id"

# Set up the API client
service = build('forms', 'v1', credentials=google.auth.default())

# Make a request to the Forms API to retrieve the form responses
response = service.forms().get(formId=form_id).execute()

# Get the list of form responses
form_responses = response['responses']

# Loop through the form responses
for form_response in form_responses:
  # Get the list of answers for the form response
  answers = form_response['answers']
  # Access the data for each answer
  for answer in answers:
    # Check the type of the answer
    if answer['type'] == 'CHECKBOX':
        # The answer is a checkbox, get the list of selected items
        selected_items = answer['checkbox']['selectedItems']
        # Access the data for each selected item
        for selected_item in selected_items:
            print(selected_item)
    else:
        print('Unknown answer type')

In [178]:
# Import the necessary libraries
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Replace with your own project ID
project_id = 'your-project-id'

# get the api_key from secrets.json
with open('./secrets.json') as json_file:
    # import all the secrets
    secrets = json.load(json_file)
    api_key = secrets['private_key_id']


# Create a service object
service = build('forms', 'v1', developerKey=api_key)

# Create a new form
form = service.forms().create(body={'title': 'My Form'}).execute()
print(f'Form created: {form["title"]} ({form["id"]})')

# Add a multiple choice question to the form
question = service.forms().questions().create(
    formId=form['id'], body={
        'checkbox': {
            'title': 'Pick Five Cards',
            'choices': [
                {'title': 'Ace of Spades'},
                {'title': 'Two of Hearts'},
                {'title': 'Three of Clubs'},
                {'title': 'Four of Diamonds'},
                {'title': 'Five of Spades'},
                {'title': 'Six of Hearts'}
            ]
        },
        'type': 'CHECKBOX'
    }).execute()
print(f'Question created: {question["title"]}')



HttpError: <HttpError 401 when requesting https://forms.googleapis.com/v1/forms?key=9aa609810e7d6339d433a8c30d4438b472d89892&alt=json returned "API keys are not supported by this API. Expected OAuth2 access token or other authentication credentials that assert a principal. See https://cloud.google.com/docs/authentication". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'CREDENTIALS_MISSING', 'domain': 'googleapis.com', 'metadata': {'method': 'google.apps.forms.v1.FormsService.CreateForm', 'service': 'forms.googleapis.com'}}]">

In [181]:
!pip install google-auth google-auth-oauthlib


In [190]:
from google.oauth2 import service_account

# Set the path to your service account key file
service_account_key_file = 'monikers-372419-9aa609810e7d.json'

# Set the scope of the API
scope = ['https://www.googleapis.com/auth/forms']
# Authenticate a credential with the service account
credentials = service_account.Credentials.from_service_account_file(
    service_account_key_file, scopes=scope)
# Construct the service object for the interacting with the Forms API
service = build('forms', 'v1', credentials=credentials)

# Create the service object
try:
    params={'key': '9aa609810e7d6339d6f93a4e96eeadbeaf21feef'}
    url = 'https://forms.googleapis.com/v1/forms'
    timeout = 5
    response = requests.get(url, params=params, headers=headers, timeout=timeout)
except requests.RequestException as e:
    print(f"Request failed: {e}")


In [195]:
# Import the required libraries
from googleapiclient.discovery import build
from google.oauth2 import service_account

# Set the path to your service account key file
service_account_key_file = 'monikers-372419-9aa609810e7d.json'

# Set the scope of the API
scope = ['https://www.googleapis.com/auth/forms']

# the form I want to replicate is:
form_url = 'https://docs.google.com/forms/d/1FaVxl33djiSvij0k-p9QfBnFbrIR_dKpC-J8Pa-7TwA'


# Authenticate a credential with the service account
credentials = service_account.Credentials.from_service_account_file(
    service_account_key_file, scopes=scope)

# Construct the service object for interacting with the Forms API
service = build('forms', 'v1', credentials=credentials)

# Send an HTTP GET request to the Forms API to retrieve a list of forms
# try:
#     response = service.forms().list().execute()
# except HttpError as e:
#     print(f"Request failed: {e}")
# except Exception as e:
#     print(f"Request failed: {e}")

# Send an HTTP request to create a new form from the template form (form_url) and set ten random cards from the deck to the options of a checkbox question, and then print the title and ID of the new form

try:
    response = service.forms().create(body={'title': 'My Form', 'source': form_url}).execute()
    print(f"Form created: {response['title']} ({response['id']})")
except HttpError as e:
    print(f"Request failed: {e}")
except Exception as e:
    print(f"Request failed: {e}")



# Check the status code of the response
if response.status_code != 200:
    print(f"Request failed: {response.status_code} {response.text}")
else:
    # Extract the list of forms from the response
    forms = response.get('forms', [])
    
    # Print the list of forms
    print(forms)



Request failed: <HttpError 403 when requesting https://forms.googleapis.com/v1/forms?alt=json returned "Request had insufficient authentication scopes.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'ACCESS_TOKEN_SCOPE_INSUFFICIENT', 'domain': 'googleapis.com', 'metadata': {'service': 'forms.googleapis.com', 'method': 'google.apps.forms.v1.FormsService.CreateForm'}}]">
Request failed: 404 <!DOCTYPE html>
<html lang=en>
  <meta charset=utf-8>
  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">
  <title>Error 404 (Not Found)!!1</title>
  <style>
    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media